## 1.0. Purpose
This blog aims to understand how to use a **classification algorithm** called **logistic regression** through **implementing an intrusion detection system**.

## 1.1. Intrusion detection
Intrusion detection is a function that detects unauthorized access to networks and hosts. According to [Wikipedia](https://en.wikipedia.org/wiki/%E4%BE%B5%E5%85%A5%E6%A4%9C%E7%9F%A5%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0), there are two main types of detection methods.

  * Anomaly type (error detection type)
  <br>Focus on **features such as packet volume and communication type**, and if there is a difference **between normal communication and** features, it is regarded as illegal.

  * Signature type
  <br>Focus on **pattern** of the packet content, and if the pattern **matches** the character string (signature) for intrusion detection prepared in advance, it is regarded as invalid.

Since we believe that the signature type has little room for machine learning, this note describes an example of implementing an **anomaly type** intrusion detection system with machine learning.

### 1.1.1. Concept
In order to implement Anomaly type intrusion detection, we **extract** festures of unauthorized access from observed target communications/data, and use them to classify communications as **normal** or **fraud**. 

<br>In addition, it is better to calculte the percentage (**probability**) of classification. Because there are communications that are difficult to judge black or white mechanically. In such a case, for example, we csn implement rules such as "If the classification probability is 70% or less, ask for human judgment" to reduce detection omissions.

This note uses logistic regression to implement an intrusion detection system with the above concept.

## 1.2. Logistic Regression
Logistic regression is an algorithm that classifies data into **predefined classes (= answers)** based on various **features** included in the data to be classified. The result of the logistic regression is **Predicted class** and **Classification probability**. 
Here, the classification probability is represented by "**a number in the range of 0 to 1**". For example, if the classification probability is 0.6, it can be regarded as "classified with 60% accuracy". 

<br>Note that logistic regression is **supervised learning** and needs to be trained in advance using a **data set** called learning data (training data).

 | Supervised Learning |
 |: -------------------------- |
 | A learning model built to make prediction, given an unforeseen input instance. Based on s training data's feature, machines learns patterns/rules of classification (e.g. "When an input with this feature is coming, then classify it as XXX" |

 | Feature |
 |: -------------------------- |
 | Numerical representation of data features. |

 | Data Set |
 |: -------------------------- |
 | A data group that pairs features and classes (answers). The quality and quantity of the data set affects the accuracy of the supervised learning classification model. Security data sets yend to include malware data sets and intrusion data sets.|

### 1.2.1. Introduction to Logistic Regression

Let's review a case using a publicly available dataset called [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set).

 | Iris flower dataset |
 |: -------------------------- |
 | The data set consists of 50 samples from each of three species of Iris (Iris Setosa, Iris virginica, and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. Often used to check the operation of machine learning algorithms. |


The Iris flower dataset contains data for three species, "setosa", "versicolor", and "virginica". Each data has **four features** - "Sepal length", "Sepal width", "Petal length", "Petal width". The table below shows part of the Iris flower dataset. From this table, it can be seen that the **varies in the width, length of petals and sepals**. For example, sepal length is in the order of "virginica > versicolor > setosa". Sepal width is in the order of “(virginica ≒ setosa) > versicolor”.

| Sepal length | Sepal width | Petal length | Petal width | Species |
|: ----------- |: ---------- |: ----------- |: ---------- |: ------ |
| 5.8 | 4.0 | 1.2 | 0.2 | setosa |
| 5.7 | 3.8 | 1.7 | 0.3 | setosa |
7.0 | 3.2 | 4.7 | 1.4 | versicolor |
6.9 | 3.1 | 4.9 | 1.5 | versicolor |
7.9 | 3.8 | 6.4 | 2.0 | virginica |
7.7 | 3.8 | 6.7 | 2.2 | virginica |


Since the length and width of sepals and petals vary depend on the species, a **model** that can classify the varieties of Iris is created by learning the **features of these Iris** with logistic regression. By giving newly collected Iris data to the model, the model can classify any species based on the learning results.

 | Model |
 |: -------------------------- |
 | Model is a result of learning from features of training data by using machine learning algorithm. |
 

The following classification results are obtained by classifying each data of the Iris flower dataset into one of three varieties using a model trained by logistic regression on **two features** - sepal length and width. Here, in logistic regression, a **line separating data** - called **Decision boundary** is calculated based on the features of the data, and each data is classified based on the line.

![Logistic regression](logistic_regression2.png)


You can see that all the data of "setosa" has been correctly classified. On the other hand, you can see that "versicolor" and "virginica" are sometimes misclassified. This misclassification occurs because data with similar feature values of "versicolor" and "virginica" exists in the data. This implifies that not all varieties with similar features can be classified straightforwardly.

In this way, logistic regression can classify each data into any class by capturing the features in the data. In this example, two features are used for simplicity, but all four features can be used.

### 1.2.2. Probability of Logistic Regression
We said that logistic regression can represent a **probability** of the prediction result, but how do you calculate this probability?

As mentioned above, logistic regression expresses the predictive result of a classification as a number between **0 and 1**. 1 can be regarded as 100%, 0.5 can be regarded as 50%, 0 can be regarded as 0%, etc. It is calculated based on the distance from the decision boundary. That is, the prediction for data far away from the decision boundary is close to 1, and the prediction for data near the decision boundary is close to 0.3. Of course, 0.5 is "50%", which means that the classificatin result is uncertain. Therefore, when implementing logistic regression, it would be better to set a threshold for the classification probability and say, "If it is 50% to 70%, humans will scrutinize the case."

In the next section, we will see the procedure and sample code of an intrusion detection system using logistic regression.

## 1.3. Implementing Intrusion Detection System
We will construct an intrusion detection system that can distinguish between normal and abnormal communication and find the type of specific attack with probability in the event of abnormality.

### 1.3.1. Preparation of Training data
Because logistic regression is supervised learning, it is necessary to prepare training data including the characteristics of normal communication and various attacking communication.

We use a publicly available data set [KDD Cup 1999 Data](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html)



| KDD Cup 1999 Data |
|: -------------------------- |
| A dataset containing traces of various intrusions simulated in a US military network environment with a large amount of normal and abnormal communication data such as intrusion and attack. 

Although the dataset is old, it contains normal communications and various types of abnormal communications, so we use it as training data for intrusion detection systems.

First, download [kddcup.data_10_percent.gz](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz) from the above site and unzip it. Then, a CSV format file “kddcup.data_10_percent_corrected” containing about 500,000 data appears. There is no column name in the data and it is difficult to handle, so download [kddcup.names](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names) containing the column names, and Add a column line to the first line of "kddcup.data_10_percent_corrected.

In this file, a Row represents one communication data. 1st to 41th column is a feature value, and 42nd column is a label (communication type). Please refer to [here](http://kdd.ics.uci.edu/databases/kddcup99/task.html) for explanation of each feature.

There are 23 types of labels. In other words, this dataset contains [24 types of communications.](https://kdd.ics.uci.edu/databases/kddcup99/training_attack_types)
An example is shown below:

| Label | Description |
|: ---------------- |: ------------ |
| normal. | Normal communication. |
| nmap. | Abnormal communication. Data at the time of Nmap execution. |
| teardrop. | Abnormal communication. Data when DoS (teardrop) is executed. |
| buffer_overflow. | Abnormal communication. Data at the time of Buffer Overflow execution. |
| guess_password. | Abnormal communication. Data when performing password guessing. |

You can see that in addition to normal communication `normal.`, abnormal communication data such as` nmap.` used for scanning and `buffer_overflow.` used for privilege escalation.

In this note, for simplicity, the labels used for learning are limited to the above five types, and all data associated with the other labels were remoced. This will reduce the number of data to around 100,000. Save the file in this state as learning data "[kddcup_train.csv](https://github.com)".

### 1.3.2. Selection of features
Select features to use for training.
The training data “kddcup_train.csv” contains 41 types of features. However, if we include the features that will not contribute to the classification of communication, the classification accuracy of the model may be decreased.

Therefore, from the 41 types of features, we select the features that are considered to characterize the five types of communication (normal, nmap, teardrop, buffer_overflow, guess_password) that are the targets of intrusion detection. This action is called **feature selection**.

| Feature Selection |
|: -------------------------- |
| A method of selecting features that will contribute to construct more accurate classification model. |

For example, if you look at the data of the label "teardrop", you can see that the feature value "wrong_fragment" is larger than other communications. In this dataset, wrong_fragment means "the number of erroneous fragments".
<br>Teardrop Attack is a type of a denial-of-service (DoS) attack which overwhelms the target machine with the incomplete data so that the victim crashes down. In Teardrop Attack, fragmented packets that are sent in the to the target machine, are buggy in nature and the victim’s machine is unable to reassemble those packets due to the bug in the TCP/IP fragmentation. In this way, the packets keep on getting accumulated over the victim’s machine and finally due to the buffer overflow, the target machine crashes down.
<br>Considering the characteristics of this attack, it would be better to select "wrong_fragment" as the teardrop feature.

In this way, the necessary features are selected one by one, taking into account the characteristics of each communication.
This time, the following features were selected.

| Label | Feature | Description |
|: ---------------- |: ------------ |: ------------ |
| nmap. | dst_host_serror_rate | SYN error rate. |
| | dst_host_same_src_port_rate | Connection rate to the same port. |
| teardrop. | wrong_fragment | Number of incorrect fragments. |
| buffer_overflow. | duration | Connection time to the host (sec). |
| | logged_in | Successful login. |
| | root_shell | Whether to acquire the root shell. |
guess_password. | dst_host_rerror_rate | REJ error rate. |
| | num_failed_logins | Number of failed login attempts. |

This time, the feature was selected based on my opinion, but there is a method of analyzing the features called **principal component analysis**. Although the analytical accuracy of this method depends on the characteristics of the target data, it is useful to find effective features by calculation.


| Principal Component Analysis (PCA) |
|: -------------------------- |
| A dimension-reduction tool that can be used to reduce a large set of variables to a small set that still contains most of the information/feature in the large set. |

Note that feature selection also has the effect of **dimension reduction**, which can improve model classification performance and reduce the amount of calculation (prevents **Curse of Dimensionality*) ).

| Dimensionality Reduction |
|: -------------------------- |
| A method of reducing the number of dimensions (number of features) in the original training data while maintaining the most of the information/feature of the data. Benefits are reduction of the amount of calculation by data compression and easier visualization. |

| Curse of Dimensionality |
|: -------------------------- |
| A problem caused by the larger number features that can represent the data. Resulting in an exponential increase in the amount of calculation. In addition, **over-learning** might be caused by using many features that are not unnecessary. |

| Overfitting |
|: -------------------------- |
| Lack og generalization ability in the model. If data not from the learning dataset is given, classification cannot be performed correctly. |

### 1.3.3. Preparation of Test data
Prepare test data to evaluate the performance of the intrusion detection system.
Again, we use the data from KDD 1999.

From [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) download and unzip [corrected.gz](http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz). Get a CSV file "corrected" containing about 300,000 data. Since the csv file also has no columns, add columns in the same way as for the training data. Then, take the data of the five types of communication (normal, nmap, teardrop, buffer_overflow, guess_password) that are targeted for intrusion detection. This will reduce the number of data to about 65,000. In this state, save the file as test data "[kddcup_test.csv](https://github.com/13o-bbr-bbq/machine_learning_security/blob/master/Security_and_MachineLearning/dataset/kddcup_test.csv)".

The training data and test data are ready.
In the next section, we will execute the sample code and verify that we can correctly classify the communications of various attacks included in the test data.

### 1.3.4. Sample code and execution result
#### 1.3.4.1. Sample code

Our overall procedure is as follows:
  1. Loading training data and test data
  2. Learning using training data (model creation)
  3. Observe the possibility of intrusion detection using test data (model evaluation)
  4. Output of evaluation results

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
import time

### Load train data

From the training data, obtain the data and label associated with the feature we selected.
In order to improve the learning accuracy, the data value of each feature is normalized (`` (X_train-X_train.mean ()) / X_train.mean () ``).

Normalization is a method of processing data values with different scales and unifying the scale of each feature so that they fall within a certain range such as 0 ~ 1. Normalisation ensure the data to be within certain range or scale, thus preventing the classification result from being greatly influenced by numerically large feature. This may improve learning accuracy.

In [2]:
df_train = pd.read_csv('kdddata_train.csv')
X_train = df_train.iloc[:, [0, 7, 10, 11, 13, 35, 37, 39]]  # feature(X)
X_train = (X_train - X_train.mean()) / X_train.mean()       # normalization
y_train = df_train.iloc[:, [41]]                            # label(y)

In [3]:
X_train
#y_train

,duration: continuous.,wrong_fragment: continuous.,num_failed_logins: continuous.,logged_in: symbolic.,root_shell: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_rerror_rate: continuous.
0,-1.0,-1.0,-1.0,0.408761,-1.0,-0.198013,-1.000000,-1.0
1,-1.0,-1.0,-1.0,0.408761,-1.0,-0.635461,-1.000000,-1.0
2,-1.0,-1.0,-1.0,0.408761,-1.0,-0.781276,-1.000000,-1.0
3,-1.0,-1.0,-1.0,0.408761,-1.0,-0.781276,-1.000000,-1.0
4,-1.0,-1.0,-1.0,0.408761,-1.0,-0.854184,-1.000000,-1.0
...,...,...,...,...,...,...,...,...
98566,-1.0,-1.0,-1.0,0.408761,-1.0,-0.927092,-1.000000,-1.0
98567,-1.0,-1.0,-1.0,0.408761,-1.0,0.239434,-1.000000,-1.0
98568,-1.0,-1.0,-1.0,0.408761,-1.0,-0.562553,16.796347,-1.0
98569,-1.0,-1.0,-1.0,0.408761,-1.0,-0.708369,10.864231,-1.0


### Load test data

In [4]:
df_test = pd.read_csv('kdddata_test.csv')
X_test = df_test.iloc[:, [0, 7, 10, 11, 13, 35, 37, 39]]
X_test = (X_test - X_test.mean()) / X_test.mean()
y_test = df_test.iloc[:, [41]]

In [5]:
X_test
#y_test

,duration: continuous.,wrong_fragment: continuous.,num_failed_logins: continuous.,logged_in: symbolic.,root_shell: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_rerror_rate: continuous.
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0
2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,3.716879,-1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0
...,...,...,...,...,...,...,...,...
65073,-1.0,-1.0,-1.0,-1.0,-1.0,-0.842771,-1.0,-1.0
65074,-1.0,-1.0,-1.0,-1.0,-1.0,-0.842771,-1.0,-1.0
65075,-1.0,-1.0,-1.0,-1.0,-1.0,-0.842771,-1.0,-1.0
65076,-1.0,-1.0,-1.0,-1.0,-1.0,-0.842771,-1.0,-1.0


### Create an instance of Logistic Regression Classifier.

'linear_model.LogisticRegression' defines a logistic regression model.
In addition,  the parameter ` C = 1e5`  sets **regularization** strength.

Regularization is a method to reduce over-learning in machine learning. [Here](https://www.analyticsvidhya.com/blog/2018/04/fundamentals-deep-learning-regularization-techniques/)'s more! 

In [6]:
logreg = linear_model.LogisticRegression(C=1e5)

##### Creating a model (execue learning)
`` 
logreg.fit (X_train, y_train)
`` 

Learning is performed by passing the feature value of `X_train` and label `y_train` as arguments of the `fit` method of `logreg`. This creates a logistic regression model.

In [7]:
start = time.perf_counter()
model = logreg.fit(X_train, y_train)
elapsed_time = time.perf_counter() - start
print('train_time   : {0}'.format(elapsed_time) + ' [sec]')

/Users/ka/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/ka/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ka/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


train_time   : 5.820506678999999 [sec]


/Users/ka/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


##### Get classification probability
`` 
probs = logreg.predict_proba(X_test)
`` 

By passing test data `X_test` as an argument of the `predict_proba` in the `logreg` model, the model classifies the test data and returns the classification probability `probs`.

In [8]:
start = time.perf_counter()
probs = logreg.predict_proba(X_test)
elapsed_time = time.perf_counter() - start
print('predict_time : {0}'.format(elapsed_time) + ' [sec]')

predict_time : 0.018595842999999945 [sec]


##### Get classification result
`` 
y_pred = logreg.predict (X_test)
`` 

By passing the test data ` X_test` as an argument of the `predict` in the `logreg` model, the model classifies the test data and returns the classification result" `y_pred`".

In [9]:
y_pred = logreg.predict(X_test)

##### Model evaluation
`` 
print ('score: {0}'. format (metrics.accuracy_score (y_test, y_pred)))
`` 

By passing the result `y_pred` and the prepared label `y_test` as arguments of the `metrics.accuracy_score` method, calculate the classification accuracy between the classification result of the model and the label.

In [10]:
print('Accuracy score: {}'.format(metrics.accuracy_score(y_test, y_pred)))

Accuracy score: 0.8992747164940533


##### Output of classification result

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
print("Accuracy score:", metrics.accuracy_score(y_test, y_pred))

                  precision    recall  f1-score   support

buffer_overflow.       0.00      0.00      0.00        22
   guess_passwd.       0.31      0.04      0.07      4367
           nmap.       0.07      1.00      0.12        84
         normal.       0.93      0.96      0.95     60593
       teardrop.       0.19      1.00      0.32        12

        accuracy                           0.90     65078
       macro avg       0.30      0.60      0.29     65078
    weighted avg       0.89      0.90      0.89     65078

Accuracy score: 0.8992747164940533


* f1_score is the harmonic mean of precision and recall. A perfect ‘f1_score’ is equal to 1.0.
* support is the number of occurrences of each class in y_test.

`` 
for predict, prob in zip (y_pred, probs):
     print ('{0} \ t {1} \ t {2}'. format (y_test.iloc [idx, [0]]. values [0], predict, np.max (prob)))
     idx + = 1
`` 

Print the outoput `Y_test` "," `y_pred`", and "` probs` one by one.

In [12]:
print('-' * 30)
print('label\tpredict\tprobability')
idx = 0
for predict, prob in zip(y_pred, probs):
    print('{0}\t{1}\t{2}'.format(y_test.iloc[idx, [0]].values[0], predict, np.max(prob)))
    idx += 1
print('finish!')

------------------------------------------------------------
label	predict	probability
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9916816688067198
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999976298423633
normal.	normal.	0.9997785210351848
normal.	normal.	0.5219228627220721
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999718275482526
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9863438169634672
normal.	normal.	0.9998045756126781
normal.	normal.	0.9591165951755115
normal.	normal.	0.9177652115118313
normal.	normal.	0.9998045756126781
normal.	normal.	0.9858309010656702
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999997202183942
nor

normal.	normal.	0.9999751858447515
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999997281563989
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	guess_passwd.	0.3333378182154334
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999996397048738
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999680099591357
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999467541831062
normal.	normal.	0.9999997307160968
normal.	normal.	0.9999781401116801
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal

normal.	normal.	0.999715515798522
normal.	normal.	0.9999781401116801
normal.	buffer_overflow.	0.7314721672610243
normal.	normal.	0.9998991584852136
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999718275482526
normal.	normal.	0.9997785210351848
normal.	normal.	0.9998088139195439
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999718275482526
normal.	normal.	0.9999996782017472
normal.	normal.	0.9999718275482526
normal.	normal.	0.9985169791515239
normal.	normal.	0.9997192874081664
normal.	normal.	0.9999112596303855
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999467541831062
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998088139195439
normal.	normal.	0.9999219049202713
normal.	normal.	0.999963670256483
normal.	normal.	0.999963670256483
normal.	normal

normal.	normal.	0.9997785210351848
normal.	normal.	0.9999587370281979
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999467541831062
normal.	buffer_overflow.	0.7314721672610243
normal.	normal.	0.9999587370281979
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998991584852136
normal.	normal.	0.9999531290965197
normal.	normal.	0.999963670256483
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999395073918076
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999683117550157
normal.	normal.	0.9997785210351848
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999997532727665
normal.	normal.	0.5571052608131801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9997807812979568
normal.	normal.	0.9999718275482526
normal.	normal.	0.9999718275482526
normal.	normal.	0.9998898878715898
normal.	normal.	0.9998045756126781
normal.	norm

normal.	normal.	0.9999718275482526
normal.	normal.	0.9999781401116801
normal.	normal.	0.9985169791515239
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999751858447515
normal.	normal.	0.999467468188759
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999680099591357
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999997483900633
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999531290965197
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998991584852136
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999112596303855
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.999

...

normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999996796389403
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998991584852136
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998991584852136
normal.	normal.	0.9998045756126781
normal.	normal.	0.9997785210351848
normal.	normal.	0.9999781401116801
normal.	normal.	0.9926542973337433
normal.	normal.	0.99

normal.	normal.	0.9998045756126781
normal.	normal.	0.9999587370281979
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9983584322690932
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999781401116801
normal.	normal.	0.9999781401116801
normal.	normal.	0.99

normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9997192874081664
normal.	normal.	0.9999467541831062
normal.	normal.	0.999963670256483
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999718275482526
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998991584852136
normal.	normal.	0.9999587370281979
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999680099591357
normal.	normal.	0.9999718275482526
normal.	normal.	0.9998045756126781
normal.	normal.	0.9999718275482526
normal.	normal.	0.9999751858447515
normal.	normal.	0.9999751858447515
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.999

normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9998045756126781
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.99

normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	nmap.	0.9576670585192378
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997785210351848
normal.	normal.	0.9997

From the results, we can say that "normal", "nmap", "teardrop", and "guess_passwd" are generally classified correctly. However, looking at the classification probabilities of "nmap", "teardrop", and "guess_passwd", the numbers are as low as "50%, 38%". This suggests that there may be more appropriate features than the ones I have selected this time. Also, it can be seen that "buffer_overflow" is hardly classified correctly and is erroneously detected as "normal" or "guess_passwd" - suggesting that the features of "buffer_overflow" need to be reviewed.

### End